In [1]:
import sys
sys.path.insert(0, 'C:/Users/Can/Dropbox/PC (2)/Desktop/My Numerical Calculations/Single Particle/Hofstadter')
from Hofstadter_Matrix_in_k_space import *

https://journals.jps.jp/doi/10.1143/JPSJ.74.1674

In [2]:
########################
###  Link Variables  ###
########################
"""
vec_1: Numpy Array
vec_2: Numpy Array
"""
def build_U(vec1, vec2):

    inner_product = np.dot(vec1, vec2.conj())
    U = inner_product / np.abs(inner_product)
    return U

In [6]:
####################################
###    Berry Curvature veya      ###
###   Lattice Field Strength     ###
###          Hesabı              ###
####################################
"""
Dk: 2D Numpy Array
"""
phi=1/q
def latF(k_vec, Dk, q):

    #(k_x, k_y)
    E, aux = np.linalg.eig(HMatrix(phi,k_vec[0], k_vec[1]))
    idx = E.argsort()
    psi = aux[:,idx] # column vectors

    #(k_x+e1, k_y)
    E, aux = np.linalg.eig(HMatrix(phi,k_vec[0]+Dk[0], k_vec[1]))
    idx = E.argsort()
    psiDx = aux[:,idx]

    #(k_x, k_y+e2)
    E, aux = np.linalg.eig(HMatrix(phi,k_vec[0], k_vec[1]+Dk[1]))
    idx = E.argsort()
    psiDy = aux[:,idx]

    #(k_x+e1, k_y+e2)
    E, aux = np.linalg.eig(HMatrix(phi,k_vec[0]+Dk[0], k_vec[1]+Dk[1]))
    idx = E.argsort()
    psiDxDy = aux[:,idx]

    U1x = np.zeros(q, dtype=complex)
    U2y = np.zeros(q, dtype=complex)
    U1y = np.zeros(q, dtype=complex)
    U2x = np.zeros(q, dtype=complex)

    for i in range(q):
        
        #U1x = U_1(k_alpha) = <u(k_alpha)|u(k_alpha+e1)> / |...|
        U1x[i] = build_U(psi[:,i], psiDx[:,i])
        
        #U2y = U_2(k_alpha) = <u(k_alpha)|u(k_alpha+e2)> / |...|
        U2y[i] = build_U(psi[:,i], psiDy[:,i])
        
        #U1y = U_1(k_alpha+e2) = <u(k_alpha+e2)|u(k_alpha+e1+e2)> / |...|
        U1y[i] = build_U(psiDy[:,i], psiDxDy[:,i])
        
        #U2x = U_2(k_alpha+e1) = <u(k_alpha+e1)|u(k_alpha+e1+e2)> / |...|
        U2x[i] = build_U(psiDx[:,i], psiDxDy[:,i])
        
    F12 = np.zeros(q, dtype=complex)

    #F12=ln(      U_1(k_alpha)*  U_2(k_alpha+e1)* 1/U_1(k_alpha+e2)*   1/U_2(k_alpha)       )
    F12 = np.log( U1x*          U2x*            1/U1y*             1/U2y              )
    # column vector
    return F12

In [7]:
# Brilliouin Zone Define
N1 = N2 = 25
Dx = (2*np.pi/q)/N1
Dy = (2*np.pi)/N2
Dk = np.array([Dx,Dy], float)

In [8]:
#############################
###  Chern Number Hesabı  ###
#############################
LF = np.zeros(q, dtype=complex)
Sum = np.zeros(q, dtype=complex)
Chern = np.zeros(q, dtype=complex)

# Brilliouin Zone Mesh
for ix in range(N1):

    kx = ix*Dx

    for iy in range(N2):

        ky = iy*Dy

        k_vec = np.array([kx,ky], float)

        LF = latF(k_vec, Dk, q)

        Sum += LF

Chern = Sum.imag/(2*np.pi)
print('q= '+str(q)+' iken '+'Bantların Chern Sayıları: '+str(Chern))

q= 5 iken Bantların Chern Sayıları: [-1. -1.  4. -1. -1.]
